In [ ]:
#| default_exp extract_acoustic

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import torch
import torchaudio
import gc

from pathlib import Path
from fastcore.script import *
from fastprogress import progress_bar, master_bar
from whisperspeech.inference import get_compute_device

In [ ]:
# unpacked small.tar should go here:
datadir = Path('/mnt/')
# you can download it downloaded from
# https://github.com/facebookresearch/libri-light/blob/main/data_preparation/README.md

# Acoustic token extraction

In [ ]:
#| export
def load(fname, newsr=24000):
    """Load an audio file to the GPU and resample to `newsr`."""
    x, sr = torchaudio.load(fname)
    _tform = torchaudio.transforms.Resample(sr, newsr)
    return _tform(x).to(get_compute_device()).unsqueeze(0)

In [ ]:
#| export
def load_model():
    "Load the pretrained EnCodec model"
    from encodec.model import EncodecModel
    model = EncodecModel.encodec_model_24khz()
    model.set_target_bandwidth(1.5)
    model.to(get_compute_device()).eval()
    return model

In [ ]:
#| export
def extract_Atoks(model, audio):
    """Extract EnCodec tokens for the given `audio` tensor (or file path)
    using the given `model` (see `load_model`)."""
    if isinstance(audio, (Path, str)):
        audio = load(audio)
    with torch.no_grad():
        frames = torch.cat([model.encode(segment)[0][0]
                            for segment in torch.split(audio, 320*20000, dim=-1)], dim=-1)
    return frames

In [ ]:
#| export
@call_parse
def extract_acoustic(
        srcdir:Path,  # source dir, should contain *.flac files
        outdir:Path,  # output dir, will get the *.encodec files
    ): 
    "Convert audio files to .encodec files with tensors of tokens"
    model = load_model()
    outdir.mkdir(exist_ok=True, parents=True)
    for name in progress_bar(list(srcdir.rglob('*.flac'))):
        outname = outdir/name.with_suffix('.encodec').name
        tokens = extract_Atoks(model, name)
        torch.save(tokens, outname)
        del tokens
        gc.collect()

In [ ]:
# process all files for speaker 1401
model = load_model()
extract_acoustic(model, datadir/'small/1401', datadir/'acoustic-1401')

In [ ]:
!du -hs {datadir}/acoustic-1401/

78M	/mnt/acoustic-1401/


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()